## 3.2.2 BM25 Score 구하기 코드 실습

In [1]:
# 설치
%pip install rank_bm25
%pip install pandas
%pip install konlpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Step1. 필요한 패키지 임포트
from rank_bm25 import BM25Okapi
import pandas as pd
import konlpy

In [ ]:
# Step2. 문서 준비
documents = [
    "고양이와 강아지", # 문서1
    "우리 강아지가 다른 강아지를 쫓아갔어", # 문서2
    "강아지는 창밖을 봐" # 문서3 
]

In [ ]:
# Step3. 한국어 토큰화를 위한 함수 작성
okt = konlpy.tag.Okt() # Okt 형태소 분석기 초기화

def tokenize_korean_text(text): # 문서 내용 토큰화 함수 정의
    return okt.morphs(text)

In [4]:
tokenize_korean_text('이 책은 RAG 시스템을 구축하는 방법에 대해 다룹니다.')

['이', '책', '은', 'RAG', '시스템', '을', '구축', '하는', '방법', '에', '대해', '다룹니다', '.']

In [5]:
# 각 문서를 토큰화 (단어 단위로 분할)
tokenize_documents=[]
for doc in documents:
    tokenize_documents.append(tokenize_korean_text(doc))
print(tokenize_documents)

[['고양이', '와', '강아지'], ['우리', '강아지', '가', '다른', '강아지', '를', '쫓아갔어'], ['강아지', '는', '창밖', '을', '봐']]


In [ ]:
# Step4. 불필요한 불용어 제거
# particles 리스트에 있는 불용어를 제거하는 함수
def delete_particles(torkenized_docs:list,
                     particles:list=["와", "가", "를", "는", "을"]):
    for sentence_list in torkenized_docs:
        for token in sentence_list:
            if token in particles:
                sentence_list.remove(token)
    return torkenized_docs

print(delete_particles(torkenized_docs=tokenize_documents))

[['고양이', '강아지'], ['우리', '강아지', '다른', '강아지', '쫓아갔어'], ['강아지', '창밖', '봐']]


In [ ]:
# Step5.  BM25 객체 생성
bm25 = BM25Okapi(tokenize_documents, k1=1.5, b=0.75)

In [ ]:
# Step6.  문서의 모든 단어를 추춯여 고유 단어 목록 생성
terms = list(set([word for doc in tokenize_documents for word in doc])) 
terms

['고양이', '쫓아갔어', '다른', '강아지', '봐', '우리', '창밖']

In [ ]:
# Step7. 각 문서에 대해 BM25 점수를 계산하여 저장
bm25_scores=[]
for term in terms:
    term_scores = bm25.get_scores([term])
    bm25_scores.append(term_scores)
bm25_scores


[array([0.62295808, 0.        , 0.        ]),
 array([0.        , 0.41700051, 0.        ]),
 array([0.        , 0.41700051, 0.        ]),
 array([0.04873883, 0.04918873, 0.04184905]),
 array([0.        , 0.        , 0.53489594]),
 array([0.        , 0.41700051, 0.        ]),
 array([0.        , 0.        , 0.53489594])]

In [10]:
# 결과를 데이터프레임으로 변환
bm25_df = pd.DataFrame(bm25_scores, index=terms, columns=['문서1', '문서2', '문서3'])

In [11]:
# 결과 출력
print("BM25 Scores Matrix:")
print(bm25_df)

BM25 Scores Matrix:
           문서1       문서2       문서3
고양이   0.622958  0.000000  0.000000
쫓아갔어  0.000000  0.417001  0.000000
다른    0.000000  0.417001  0.000000
강아지   0.048739  0.049189  0.041849
봐     0.000000  0.000000  0.534896
우리    0.000000  0.417001  0.000000
창밖    0.000000  0.000000  0.534896
